## Import covariate modeling results, correct p-values

### Load modules, set paths, load files:

In [1]:
import pandas as pd
import scanpy as sc
from statsmodels.stats.multitest import multipletests
import numpy as np
import os

optional, for pretty code formatting:

In [2]:
%load_ext lab_black

set paths:

In [3]:
path_celltype_df = "../../supporting_files/celltype_structure_and_colors/manual_anns_grouped_order_and_colors.csv"
dir_modeling_input = "../../results/covariate_modeling/input/"
dir_modeling_output = "../../results/covariate_modeling/output/"

load files:

In [4]:
cts_ordered_df = pd.read_csv(path_celltype_df, index_col=0)

get celltype names in bio order:

In [5]:
cts = cts_ordered_df.index.tolist()

### import gene modeling results and perform multiple-testing-correction

store top 100 genes in new file (set to False if not wanted)

In [6]:
store_top_100 = False

prepare:

In [7]:
# . create empty dictionaries to store results in:
res_per_ct = dict()
# list covariate names of interest
covs = [
    "anatomical_region_ccf_score",
    "nose",
    "sexfemale",
    "age",
    "BMI",
    "ethnicity",
    "smoking_status_num",
]
# create empty dataframe in which we will store the number of significant
# genes per covariate-celltype pair
n_genes_sign = pd.DataFrame(index=cts, columns=covs + ["total_n_genes"])
# set significance threshold
alpha = 0.05

Now get results for every cell type. We will add gene names to modeling output tables here (these were for some reason not added to R output, but we can copy them from the modeling input), and we will perform multiple-testing correction. We'll also store the number of significant genes per covariate-cell-type pair. Finally, we'll store the results for the top 100 genes in a separate table.<br>
Consider storing all results (now with gene names and adjusted p-values), not done at the moment.

In [8]:
# now loop through cell types and import results
for celltype in cts:
    print(celltype)
    # replace spaces with underscores for file reading
    ct_no_spaces = celltype.replace(" ", "_")
    file_path = os.path.join(dir_modeling_output, f"{ct_no_spaces}/mm_output.tsv")
    # there are only results files for cell types that were modeled. Cell types with
    # e.g. too few samples weren't modeled, and we will therefore not import
    # any results for those.
    if os.path.isfile(file_path):
        res = pd.read_csv(file_path, sep="\t")
        # store information about number of significant genes.
        # perform p-value correction
        total_n_genes = res.shape[0]
        n_genes_sign.loc[celltype, "total_n_genes"] = total_n_genes
        # get names of all covariates included, based on the loaded results
        for col in res.columns:
            if col.startswith("P.value"):
                cov_name = col.replace("P.value.", "")
                if cov_name != "(Intercept)":
                    # correct p-values and store number of significant genes:
                    res[f"adj_{col}"] = multipletests(res[col].values, method="fdr_bh")[
                        1
                    ]
                    n_sign = (res[f"adj_{col}"] < alpha).sum()
                    n_genes_sign.loc[celltype, cov_name] = n_sign

        # import matching file for gene names from input file (instead of output file):
        # for some reason R didn't store the gene names for the output...
        with open(
            os.path.join(dir_modeling_input, f"{ct_no_spaces}/sample_gene_sums.csv"),
            "r",
        ) as f:
            gene_names = f.readline().split(",")
        if gene_names[0] == "sample":
            gene_names = gene_names[1:]
        # check if gene names length corresponds to results file:
        if len(gene_names) != res.shape[0]:
            raise ValueError(
                "Length of gene names does not correspond to results number of rows."
            )
        else:
            res.index = gene_names
        if store_top_100:
            # create dataframe to store results in:
            top100 = pd.DataFrame(index=range(100))
            # store which covariates were modeled for this celltype:
            covariates = [
                col.lstrip("Coef.")
                for col in res.columns
                if "Coef." in col and col != "Coef.(Intercept)"
            ]
            for cov in covariates:
                topres = (
                    res.sort_values(
                        by=[f"adj_P.value.{cov}", f"P.value.{cov}"], ascending=True
                    )
                    .loc[:, [f"Coef.{cov}", f"adj_P.value.{cov}"]]
                    .iloc[:100, :]
                )
                topres[f"gene.{cov}"] = topres.index
                # move gene to left side of dataframe:
                topres = topres.iloc[:, [2, 0, 1]]
                topres.index = range(100)
                top100 = pd.concat((top100, topres), axis=1)
                # write result:
                top100.to_csv(
                    os.path.join(
                        dir_modeling_output,
                        f"{ct_no_spaces}/{ct_no_spaces}_top100genes.csv",
                    )
                )
        # store results in dictionary:
        res_per_ct[celltype] = res
        # and write to file:
        res.to_csv(
            os.path.join(dir_modeling_output, f"{ct_no_spaces}/mm_output_padj.csv")
        )
    else:
        print("No results for", celltype)

Basal
Multiciliated
Secretory
Transitional Club-AT2
Ionocyte
Tuft
No results for Tuft
Neuroendocrine
No results for Neuroendocrine
SMG serous
SMG mucous
No results for SMG mucous
SMG duct
AT1
AT2
EC arterial
EC capillary
EC venous
Lymphatic EC
Peribronchial fibroblasts
Adventitial fibroblasts
Alveolar fibroblasts
Pericytes
Subpleural fibroblasts
No results for Subpleural fibroblasts
Myofibroblasts
No results for Myofibroblasts
Smooth muscle
Fibromyocytes
No results for Fibromyocytes
Mesothelium
No results for Mesothelium
B cell
Plasma cells
T cell lineage
Innate lymphoid cell NK
DC1
No results for DC1
DC2
Migratory DCs
No results for Migratory DCs
Plasmacytoid DCs
Alveolar macrophages
Monocyte-derived Mφ
Interstitial Mφ perivascular
Monocytes
Mast cells


print table with number of significant results per celltype-covariate pair, to get a feeling for our overall results:

In [9]:
n_genes_sign

,anatomical_region_ccf_score,nose,sexfemale,age,BMI,ethnicity,smoking_status_num,total_n_genes
Basal,4511,8944,47,1,0,NaN,4,20936
Multiciliated,349,2794,36,5,0,NaN,0,19423
Secretory,281,8072,17,1,0,NaN,0,19636
Transitional Club-AT2,NaN,NaN,NaN,0,0,NaN,0,12115
Ionocyte,4,0,0,0,2,NaN,0,8013
Tuft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Neuroendocrine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SMG serous,0,25,2,0,0,NaN,1,8044
SMG mucous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SMG duct,52,103,6,0,0,NaN,0,11755


Create summary statistics: number of significant genes for demographic covariates, and for anatomical location covariates:

In [10]:
n_genes_1d_demo = (
    n_genes_sign.loc[  # exclude total_n_genes column and anatomical location
        :,
        [
            "sexfemale",
            "age",
            "BMI",
            "ethnicity",
            "smoking_status_num",
        ],
    ].unstack()
)
n_genes_1d_ana = (
    n_genes_sign.loc[  # exclude total_n_genes column and anatomical location
        :,
        [
            "anatomical_region_ccf_score",
            "nose",
        ],
    ].unstack()
)
n_cts = n_genes_sign.dropna(axis=0, how="all").shape[0]

Print summary stats:

In [11]:
print(
    "Total significant associations with demographic covariates:", n_genes_1d_demo.sum()
)
print(
    "Total significant associations with anatomical region covariates:",
    n_genes_1d_ana.sum(),
)
print("Total modeled cts:", n_cts)

Total significant associations with demographic covariates: 936
Total significant associations with anatomical region covariates: 25468
Total modeled cts: 29
